**TC2008B Modelación de sistemas multiagentes con gráficas computacionales (Gpo 104)**<br>

*Actividad Integradora | Parte 1. Sistemas multiagentes*<br>


Profesor del modulo:<br>
Jorge Mario Cruz Duarte<br>

Integrantes del Equipo:<br>

Miranda Magallanes García		    A00832477<br>
Miguel Ángel Bermea Rodríguez		A01411671<br>
Gustavo Luna Muñoz                  A01411619<br>
Diego Alonso Bugarin Estrada	    A01620485<br>

In [22]:
from mesa import Agent, Model 
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2 ** 128

import numpy as np
import pandas as pd

import time
import datetime
import random

movements = 0

*AGENTES*

In [23]:
from hashlib import new


class CellAgent(Agent):
    """
    Representa a un agente o una celda con estado:
    0 | Vacio
    1 | Caja
    """
    def __init__(self, unique_id, model, state):
        super().__init__(unique_id, model)
        self.state = state
        self.next_state = None
    
    def step(self):
        if self.next_state == 0:
            self.next_state = 1

        elif self.next_state == 1: 
            self.state = 0
            self.next_state = None

class RobotAgent(Agent):
    """
    Representa al robot que recorrera el grid y organizara las cajas
    """
    def __init__(self, unique_id, model, pos, width, box_id):
        super().__init__(unique_id, model)
        self.next_state = None
        self.pos = pos
        self.state = 2  
        self.count_box = 0  
        self.count_storage = 0
        self.next_line = self.pos[1] 
        self.width = width
        self.returning = False
        self.done = None
        self.box_id = box_id

    def step(self):
        empty_neighbours = []
        box_neighbours = []
        robot_neighbours = []
        global movements

        neighbours = self.model.grid.get_neighbors(
            self.pos,
            moore=False,
            include_center=False)

        for neighbor in neighbours:
            if(neighbor.state == 1 and neighbor.pos[0] > self.pos[0] and neighbor.pos[1] == self.next_line):
                if(neighbor.pos == self.pos):
                    box_neighbours.insert(0, neighbor)
                else:
                    box_neighbours.append(neighbor)
            elif(neighbor.state == 0 and neighbor.pos[1] == self.pos[1]): 
                empty_neighbours.append(neighbor)
            elif(neighbor.state == 2): 
                robot_neighbours.append(neighbor)

        if self.done == True: 
            new_position = self.pos

        elif box_neighbours and self.count_box < 1: 
            box_neighbours[0].state = 0
            self.count_box += 1
            new_position = self.pos

        elif self.pos[1] != self.next_line: 
            new_position = (self.pos[0], self.pos[1] + 1)
            movements+=1

        elif self.count_box == 1 and self.pos[0] != 1: 
            new_position = (self.pos[0] - 1, self.pos[1])
            movements+=1

        elif self.count_box == 1 and self.pos[0] == 1: 
            if self.returning == False: 
                a = CellAgent(self.box_id, self, state=1)
                self.box_id += 1
                self.model.grid.place_agent(a, (0, self.pos[1]))
                self.model.schedule.add(a)

            self.count_box = 0
            self.count_storage += 1

            if self.count_storage == 5 or self.returning == True: 
                if (self.pos[1] + 5) <= (self.width - 1):
                    self.next_line = self.pos[1] + 5
                    self.count_storage = 0
                    new_position = (1, self.pos[1] + 1)
                    movements+=1
                    self.returning = False
                else:
                    print(self.count_storage)
                    new_position = self.pos
                    self.done = True
                    self.model.done_count += 1
            else: 
                new_position = (self.pos[0] + 1, self.pos[1])
                movements+=1
                
        elif self.pos[0] == self.width - 1: 
            if (self.pos[1] + 5) <= (self.width - 1):
                    self.count_box = 1
                    self.returning = True
                    new_position = (self.pos[0] - 1, self.pos[1])
                    movements+=1
            else: 
                new_position = self.pos
                self.done = True
                self.model.done_count += 1
                
        elif empty_neighbours: 
            try: 
                new_position = empty_neighbours[-1].pos
                movements+=1
            except: 
                new_position = self.pos

        self.model.grid.move_agent(self, new_position)
        
        """else: 
            try: 
                rdm = random.randint(0, len(robot_neighbours) - 1)
                new_position = robot_neighbours[rdm].pos
            except: 
                new_position = self.pos"""
        
            
            



*MODELO*

In [24]:
class SimulationModel(Model):
    """
    Modelo 
    """
    def __init__(self, width, height, num_robot):
        self.num_agents = width * height
        self.grid = MultiGrid(width, height, False)
        self.schedule = SimultaneousActivation(self)
        self.done_count = 0

        emptycells = [(x, y) for _, x, y in self.grid.coord_iter()]
        boxes = random.randint(int(width*5*.6), width*5)
        #boxcells = random.sample(emptycells, int(percentage*self.num_agents))
        boxcells = random.sample(emptycells, boxes)


        for (content, x, y) in self.grid.coord_iter():

            a = CellAgent((x, y), self, state=1)
            if ((x, y) in boxcells and (x != 0 and x != 1)):
                a.state = 1
                self.grid.place_agent(a, (x, y))
                self.schedule.add(a)

            elif (x == 0): 
                a.state = 3
                self.grid.place_agent(a, (x, y))
                self.schedule.add(a)
                
            else:
                a.state = 0
                self.grid.place_agent(a, (x, y))
                self.schedule.add(a)
        
        for i in range (num_robot):
            box_id_ = i*width + width**2
            b = RobotAgent(i, self, (1, i), width, box_id_)
            self.grid.place_agent(b, (1, i)) 
            self.schedule.add(b) 

        # Aquí definimos el colector de datos para obtener el grid completo.
        self.datacollector = DataCollector(
            model_reporters={"Grid": self.get_grid}
        )

    def step(self):
        """
        En cada paso el colector toma la información que se definió y almacena el grid para luego
        graficarlo.
        """
        self.datacollector.collect(self)
        self.schedule.step()

    def get_grid(self):
        """
        Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
        """

        # Generamos la grid para contener los valores
        grid = np.zeros((self.grid.width, self.grid.height))

        # Asignamos una celda a cada uno de los elementos de la grilla
        for cell in self.grid.coord_iter():
            cell_content, x, y = cell
            for obj in cell_content:
                if isinstance(obj, RobotAgent):
                    grid[x][y] = 2
                else:
                    grid[x][y] = obj.state
        return grid

In [25]:
# Definimos el tamaño del Grid
GRID_SIZE = int(input("Tamaño del grid: "))

# Definimos el número de generaciones a correr
NUM_GENERATIONS = int(input("Número de generaciones: "))

# Registramos el tiempo de inicio y corremos el modelo
start_time = time.time()
model = SimulationModel(GRID_SIZE, GRID_SIZE, 5)
k = 0
counter = 0
while k < NUM_GENERATIONS:
    k+=1
    model.step()
    if(1 not in model.datacollector.get_model_vars_dataframe().loc[[counter]].values[0][0]):
        break
    counter+=1
    if model.done_count == 5: 
        k = NUM_GENERATIONS


# Tiempo necesario hasta que todas las cajas están en pilas de máximo 5 cajas.
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

# Número de movimientos realizados por todos los robots. 
print("Número de movimientos realizados por todos los robots: ", movements)

Tiempo de ejecución: 0:00:00.078780
Número de movimientos realizados por todos los robots:  339


In [26]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [27]:
%%capture

from typing import Counter


cmap = matplotlib.cm.get_cmap('viridis', 4)
cmap = cmap(np.linspace(0, 1, 4))

cmap[0] = np.array([255/255, 255/256, 255/256, 1]) #  Piso
cmap[1] = np.array([128/256, 64/256, 0/256, 1]) # Caja
cmap[2] = np.array([37/256, 40/256, 80/256, 1]) # Robot
cmap[3] = np.array([156/256, 156/256, 156/256, 1]) # Estante

new_cmap = matplotlib.colors.ListedColormap(cmap)

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=new_cmap)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=counter)


In [28]:
anim